In [4]:
import os
import cv2

def draw_bounding_box(image, box, class_id, class_names, color=(255, 0, 0), box_thickness=1, font_scale=0.5, font_thickness=1):
    """
    Рисует bounding box на изображении.
    
    image: изображение в формате numpy array
    box: кортеж (x_center, y_center, width, height) в формате YOLO
    class_id: идентификатор класса
    class_names: список имен классов
    color: цвет bounding box и текста (по умолчанию синий)
    box_thickness: толщина линий bounding box
    font_scale: масштаб шрифта для текста
    font_thickness: толщина шрифта для текста
    """
    height, width, _ = image.shape
    x_center, y_center, w, h = box
    
    # Преобразование из нормализованных координат YOLO в пиксели
    xmin = int((x_center - w / 2) * width)
    xmax = int((x_center + w / 2) * width)
    ymin = int((y_center - h / 2) * height)
    ymax = int((y_center + h / 2) * height)
    
    # Убедимся, что координаты находятся в пределах изображения
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    xmax = min(width, xmax)
    ymax = min(height, ymax)
    
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, int(box_thickness))
    
    label = f"{class_names[class_id]}"
    (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, int(font_thickness))
    
    # Рисуем фон для текста
    cv2.rectangle(image, (xmin, ymin - label_height - baseline), (xmin + label_width, ymin), color, cv2.FILLED)
    # Рисуем текст
    cv2.putText(image, label, (xmin, ymin - baseline), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), int(font_thickness))

def visualize_annotations(image_path, annotation_path, class_names, color=(255, 0, 0), box_thickness=1, font_scale=0.5, font_thickness=1):
    """
    Визуализирует аннотации на изображении.
    
    image_path: путь к изображению
    annotation_path: путь к YOLO файлу аннотаций
    class_names: список имен классов
    color: цвет bounding box и текста (по умолчанию синий)
    box_thickness: толщина линий bounding box
    font_scale: масштаб шрифта для текста
    font_thickness: толщина шрифта для текста
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image {image_path}")
        return None
    
    try:
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Skipping invalid annotation in {annotation_path}: {line.strip()}")
                    continue
                class_id = int(parts[0])
                box = tuple(map(float, parts[1:]))
                draw_bounding_box(image, box, class_id, class_names, color, box_thickness, font_scale, font_thickness)
    except Exception as e:
        print(f"Error processing {annotation_path}: {e}")
        return None
    
    return image  # Возвращаем изображение с аннотациями

def process_directory(images_dir, annotations_dir, output_dir, class_names, color=(255, 0, 0), box_thickness=1, font_scale=0.5, font_thickness=1):
    """
    Обрабатывает все изображения в директории и визуализирует аннотации.
    
    images_dir: директория с изображениями
    annotations_dir: директория с YOLO файлами аннотаций
    output_dir: директория для сохранения визуализированных изображений
    class_names: список имен классов
    color: цвет bounding box и текста (по умолчанию синий)
    box_thickness: толщина линий bounding box
    font_scale: масштаб шрифта для текста
    font_thickness: толщина шрифта для текста
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    image_files.sort()  # Сортируем для сохранения порядка

    for image_name in image_files:
        image_path = os.path.join(images_dir, image_name)
        annotation_name = f"{os.path.splitext(image_name)[0]}.txt"
        annotation_path = os.path.join(annotations_dir, annotation_name)
        
        if os.path.exists(annotation_path):
            print(f"Processing {image_name}")
            annotated_image = visualize_annotations(image_path, annotation_path, class_names, color, box_thickness, font_scale, font_thickness)
            
            if annotated_image is not None:
                output_image_path = os.path.join(output_dir, image_name)
                cv2.imwrite(output_image_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
        else:
            print(f"Annotation not found for {image_name}")

# Пример использования:


# Пример использования:



In [5]:
images_dir = '/home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/images'  # Замените на путь к директории с изображениями
annotations_dir = '/home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_copy_2/labels'  # Замените на путь к директории с YOLO аннотациями
output_dir = '//home/andrew/works/shpad_add/Detection-of-welding-seams/dataset_risovashka'  # Замените на путь к директории для сохранения визуализированных изображений
class_names = ['adj', 'int', 'geo', 'pro', 'non']

# Параметры визуализации
color = (255, 0, 0)  # Цвет для bounding box и текста (синий)
box_thickness = 2.5   # Толщина линий bounding box
font_scale = 1.5    # Масштаб шрифта для текста
font_thickness = 2   # Толщина шрифта для текста



In [6]:
process_directory(images_dir, annotations_dir, output_dir, class_names, color, box_thickness, font_scale, font_thickness)

Processing 1 (1).jpg
Processing 1 (10).jpg
Processing 1 (11).jpg
Processing 1 (12).jpg
Processing 1 (13).jpg
Processing 1 (2).jpg
Processing 1 (3).jpg
Processing 1 (4).jpg
Processing 1 (5).jpg
Processing 1 (6).jpg
Processing 1 (7).jpg
Processing 1 (8).jpg
Processing 1 (9).jpg
Processing 11 (1).jpg
Processing 11 (10).jpg
Processing 11 (11).jpg
Processing 11 (12).jpg
Processing 11 (13).jpg
Processing 11 (14).jpg
Processing 11 (15).jpg
Processing 11 (16).jpg
Processing 11 (17).jpg
Processing 11 (18).jpg
Processing 11 (19).jpg
Processing 11 (2).jpg
Processing 11 (20).jpg
Processing 11 (21).jpg
Processing 11 (22).jpg
Processing 11 (23).jpg
Processing 11 (24).jpg
Processing 11 (25).jpg
Processing 11 (26).jpg
Processing 11 (27).jpg
Processing 11 (28).jpg
Processing 11 (29).jpg
Processing 11 (3).jpg
Processing 11 (30).jpg
Processing 11 (31).jpg
Processing 11 (32).jpg
Processing 11 (33).jpg
Processing 11 (34).jpg
Processing 11 (35).jpg
Processing 11 (36).jpg
Processing 11 (37).jpg
Processing 11